<a href="https://colab.research.google.com/github/shaguftah10sep/assignments/blob/main/python_retail_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from google.colab import files
uploaded = files.upload()


# Load all files
transactions = pd.read_parquet('transactions.parquet')
vip_mapping = pd.read_csv('umd_vip_to_profile_mapping.csv', sep=';')
vips_01 = pd.read_csv('vips_2020-11-01.csv')
vips_15 = pd.read_csv('vips_2020-11-15.csv')
vips_25 = pd.read_csv('vips_2020-11-25.csv')

#  Filter the latest VIP snapshot for "The Netherlands
vips_netherlands = vips_25[vips_25['country'] == 'The Netherlands']

# Join VIPs to profiles using the mapping file
vip_sales_mapping = vip_mapping[vip_mapping['active'] == 'yes']
vips_netherlands = vips_netherlands.merge(vip_sales_mapping, on='vip_id', how='left')

# Join transactions with VIPs using profile_id
vip_transactions = vips_netherlands.merge(transactions, on='profile_id', how='left')

# Calculate total sales value
vip_transactions['net_price_per_unit'] = vip_transactions['recommended_retail_price_per_unit'] - vip_transactions['discount_amount_per_unit'].fillna(0)
vip_transactions['total_sales_value'] = vip_transactions['net_price_per_unit'] * vip_transactions['quantity']

# Group by VIP to get total sales per VIP
vip_sales_summary = vip_transactions.groupby(['vip_id', 'first_name', 'last_name', 'email']).agg({
    'total_sales_value': 'sum'
}).reset_index()

# Fill missing total_sales_value with 0 for VIPs without purchases
vip_sales_summary['total_sales_value'].fillna(0, inplace=True)

# Sort by total sales value in descending order
vip_sales_summary = vip_sales_summary.sort_values(by='total_sales_value', ascending=False)

# Output final result
print(vip_sales_summary[['first_name', 'last_name', 'email', 'total_sales_value']])


Saving transactions.parquet to transactions (4).parquet
Saving umd_vip_to_profile_mapping.csv to umd_vip_to_profile_mapping (4).csv
Saving vips_2020-11-01.csv to vips_2020-11-01 (3).csv
Saving vips_2020-11-15.csv to vips_2020-11-15 (3).csv
Saving vips_2020-11-25.csv to vips_2020-11-25 (3).csv
  first_name      last_name                     email  total_sales_value
1    Brandon         Ritter  Brandon.Ritter@gmail.com             1951.0
0    Kristin  Brooks-Ritter        Kristin@Brooks.com              618.0
2       Seth            Lee              Seth@Lee.com                0.0
3      Edwin          Floyd     Edwin.Floyd@gmail.com                0.0
